### CUDA 버전 확인하기

- !nvidia-smi
- !nvcc --version

둘의 차이점은
nvidia-smi는 해당 장치에서 설치 가능한 가장 높은 버전을 보여주고,
nvcc --version은 현재 설치된 cuda 버전을 보여줌

출처 : https://stackoverflow.com/questions/9727688/how-to-get-the-cuda-version

In [1]:
gpu_info = !nvcc --version

In [2]:
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [4]:
!nvidia-smi

Thu Jul 20 04:55:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# PyTorch 2.x 버전 설치
try:
    # 기본적으로 https://pytorch.kr/get-started/locally/ 에서
    # cuda 버전과 패키지매니저에 맞는 설치 명령어를 확인 가능
    # %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
    import torch
    print(torch.__version__)
except:
    pass

2.0.1+cu118


----------------------------------------------------------

# VGGNet 직접 구현하기

In [6]:
# pytorch에서 사용할 함수들 호출하기
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [7]:
# 데이터가 상당히 크기 때문에 시간이 좀 소요됨
transform = transforms.Compose([
    transforms.Resize(72),      # image 사이즈 조정
    transforms.RandomCrop(56),  # image 자리
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5 ), (0.5, 0.5, 0.5 )),
])

train_dataset = datasets.STL10(root='./data', split='train', download=True, transform=transform)
# train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)

test_dataset = datasets.STL10(root='./data', split='test', download=True, transform=transform)
# test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False)

100%|██████████| 2640397119/2640397119 [04:58<00:00, 8854871.02it/s]


Extracting ./data/stl10_binary.tar.gz to ./data
Files already downloaded and verified


### 1) VGGNet11 구현

In [23]:
class VGGNet11(nn.Module):

    def __init__(self, num_classes=10):
        super(VGGNet11, self).__init__()
        self.convnet = nn.Sequential(
            # 여기에 CNN 모델을 구현해주세요.
            # input image size = 3x56x56
            nn.Conv2d(3, 64, 3, 1, 1),
            # 64x56x56
            nn.ReLU(True),
            nn.MaxPool2d(2),

            # 64x28x28
            nn.Conv2d(64, 128, 3, 1, 1),
            # 128x28x28
            nn.ReLU(True),
            nn.MaxPool2d(2),

            # 128x14x14
            nn.Conv2d(128, 256, 3, 1, 1),
            # 256x14x14
            nn.ReLU(True),
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.ReLU(True),
            nn.MaxPool2d(2),

            # 256x7x7
            nn.Conv2d(256, 512, 3, 1, 1),
            # 512x7x7
            nn.ReLU(True),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.ReLU(True),
            nn.MaxPool2d(2),

            # 512x3x3
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.ReLU(True),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.ReLU(True),
            nn.MaxPool2d(2)) # 512x1x1

        self.fclayer = nn.Sequential(
            # 여기에 FC 모델을 구현해주세요.
            nn.Linear(512, num_classes),
            # nn.Softmax()
        )

    def forward(self, x):
        x = self.convnet(x)
        x = torch.flatten(x, 1)
        output = self.fclayer(x)
        return output

In [25]:
model = VGGNet11(num_classes=10)

In [26]:
# 학습을 위한 설정
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(model.parameters(),lr=1e-5)
scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

In [27]:
# 모델 학습
epochs = 40
dry_run = False # 1 배치만 훈련

for epoch in range(1, epochs+1):
    # 학습
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.detach().cpu().item()))
            if dry_run:
                break

    # 테스트
    model.eval()
    test_loss = 0
    correct = 0

    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        with torch.no_grad():
            output = model(data)
        test_loss += criterion(output, target).detach().cpu().item()
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    scheduler.step()


Train Epoch: 1 [0/5000 (0%)]	Loss: 2.305196
Train Epoch: 1 [2560/5000 (50%)]	Loss: 2.302308

Test set: Average loss: 0.0092, Accuracy: 800/8000 (10%)

Train Epoch: 2 [0/5000 (0%)]	Loss: 2.302139
Train Epoch: 2 [2560/5000 (50%)]	Loss: 2.300629

Test set: Average loss: 0.0092, Accuracy: 800/8000 (10%)

Train Epoch: 3 [0/5000 (0%)]	Loss: 2.300173
Train Epoch: 3 [2560/5000 (50%)]	Loss: 2.297762

Test set: Average loss: 0.0092, Accuracy: 906/8000 (11%)

Train Epoch: 4 [0/5000 (0%)]	Loss: 2.288206
Train Epoch: 4 [2560/5000 (50%)]	Loss: 2.268679

Test set: Average loss: 0.0089, Accuracy: 1614/8000 (20%)

Train Epoch: 5 [0/5000 (0%)]	Loss: 2.236389
Train Epoch: 5 [2560/5000 (50%)]	Loss: 2.157365

Test set: Average loss: 0.0084, Accuracy: 1682/8000 (21%)

Train Epoch: 6 [0/5000 (0%)]	Loss: 2.088343
Train Epoch: 6 [2560/5000 (50%)]	Loss: 2.041504

Test set: Average loss: 0.0082, Accuracy: 1792/8000 (22%)

Train Epoch: 7 [0/5000 (0%)]	Loss: 2.080615
Train Epoch: 7 [2560/5000 (50%)]	Loss: 2.076967

VGGNet11의 경우 35%의 Accuracy로 분류했다.

In [22]:
for name, param in model.named_parameters():
    print(name)
    print(param)
    break

convnet.0.weight
Parameter containing:
tensor([[[[-0.1144, -0.1549,  0.1741],
          [ 0.0975, -0.1651,  0.0737],
          [ 0.1242, -0.1170,  0.1480]],

         [[-0.0786,  0.1477, -0.0866],
          [-0.1769, -0.0458,  0.1403],
          [ 0.1384,  0.1054,  0.1736]],

         [[ 0.0095,  0.0180,  0.0244],
          [-0.1013, -0.0465,  0.0473],
          [ 0.1785,  0.1669,  0.1757]]],


        [[[-0.1443, -0.0752,  0.0551],
          [ 0.1028,  0.0061,  0.1706],
          [-0.1726,  0.0477, -0.0700]],

         [[-0.1586,  0.0794, -0.1713],
          [ 0.0256,  0.1517,  0.1503],
          [ 0.0581,  0.0465, -0.0184]],

         [[ 0.0417, -0.1764,  0.1138],
          [ 0.0143,  0.1641,  0.1871],
          [-0.1025, -0.0509, -0.1493]]],


        [[[ 0.0880,  0.0897,  0.1096],
          [-0.1456,  0.0789,  0.1025],
          [-0.0695,  0.0681, -0.1109]],

         [[ 0.1342,  0.1623,  0.0980],
          [ 0.1525,  0.1157,  0.0131],
          [-0.0897,  0.1386, -0.1919]],

     

---

### 2) VGGNet13 구현

In [37]:
# VGGNet13 구현
class VGGNet13(nn.Module):
    def __init__(self, num_classes=10):
        super(VGGNet13, self).__init__()
        self.convnet = nn.Sequential(
            # 3x56x56
            nn.Conv2d(3, 64, 3, 1, 1),
            # 64x56x56
            nn.ReLU(True),
            nn.Conv2d(64, 64, 3, 1, 1),
            nn.ReLU(True),
            nn.MaxPool2d(2),

            # 64x28x28
            nn.Conv2d(64, 128, 3, 1, 1),
            # 128x28x28
            nn.ReLU(True),
            nn.Conv2d(128, 128, 3, 1, 1),
            nn.ReLU(True),
            nn.MaxPool2d(2),

            # 128x14x14
            nn.Conv2d(128, 256, 3, 1, 1),
            # 256x14x14
            nn.ReLU(True),
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.ReLU(True),
            nn.MaxPool2d(2),

            # 256x7x7
            nn.Conv2d(256, 512, 3, 1, 1),
            # 512x7x7
            nn.ReLU(True),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.ReLU(True),
            nn.MaxPool2d(2),

            # 512x3x3
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.ReLU(True),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.ReLU(True),
            nn.MaxPool2d(2))    # 512x1x1

        self.fclayer = nn.Sequential(
            nn.Linear(512, num_classes),
        )
    def forward(self, x):
        x = self.convnet(x)
        x = torch.flatten(x, 1)
        output = self.fclayer(x)
        return output

In [38]:
model = VGGNet13(num_classes=10)

In [39]:
# 학습을 위한 설정
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(model.parameters(),lr=1e-5)
scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

In [40]:
# 모델 학습
epochs = 40
dry_run = False # 1 배치만 훈련

for epoch in range(1, epochs+1):
    # 학습
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.detach().cpu().item()))
            if dry_run:
                break

    # 테스트
    model.eval()
    test_loss = 0
    correct = 0

    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        with torch.no_grad():
            output = model(data)
        test_loss += criterion(output, target).detach().cpu().item()
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    scheduler.step()


Train Epoch: 1 [0/5000 (0%)]	Loss: 2.301809
Train Epoch: 1 [2560/5000 (50%)]	Loss: 2.303134

Test set: Average loss: 0.0092, Accuracy: 800/8000 (10%)

Train Epoch: 2 [0/5000 (0%)]	Loss: 2.301107
Train Epoch: 2 [2560/5000 (50%)]	Loss: 2.300568

Test set: Average loss: 0.0092, Accuracy: 800/8000 (10%)

Train Epoch: 3 [0/5000 (0%)]	Loss: 2.303256
Train Epoch: 3 [2560/5000 (50%)]	Loss: 2.302809

Test set: Average loss: 0.0092, Accuracy: 1442/8000 (18%)

Train Epoch: 4 [0/5000 (0%)]	Loss: 2.303887
Train Epoch: 4 [2560/5000 (50%)]	Loss: 2.300164

Test set: Average loss: 0.0092, Accuracy: 1440/8000 (18%)

Train Epoch: 5 [0/5000 (0%)]	Loss: 2.295112
Train Epoch: 5 [2560/5000 (50%)]	Loss: 2.287615

Test set: Average loss: 0.0090, Accuracy: 1403/8000 (18%)

Train Epoch: 6 [0/5000 (0%)]	Loss: 2.259987
Train Epoch: 6 [2560/5000 (50%)]	Loss: 2.181263

Test set: Average loss: 0.0085, Accuracy: 1479/8000 (18%)

Train Epoch: 7 [0/5000 (0%)]	Loss: 2.105456
Train Epoch: 7 [2560/5000 (50%)]	Loss: 2.04976

VGGNet13의 경우 35%의 Accuracy로 분류했다.